# Project: Data Modeling with Cassandra
This project presents an ETL pipeline using Python to extract user activity data on a startup music app, `Sparkify`. The data reside in a directory of CSV files, which are being loaded to an Apache Cassandra keyspace, `sparkifyks`.  The data is modelled to answer the queries below;

* get the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
* get the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
* get every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

In Cassandra, every table represents a query, therefore the data is modelled into 3 tables to help answer the above queries.

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

from cassandra.cluster import Cluster

#### List of filepaths to original event csv data files partitioned by date.

In [2]:
# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# list of paths to all event csv files
event_filepaths = []

# For loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*.csv'))
    for file in file_path_list:
        event_filepaths.append(os.path.abspath(file))
        
print(event_filepaths)

['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/work

#### Processing the files to create the data file csv that will be used for the Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in event_filepaths:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

            
# creating an "all_event_datafiles" csv file containing all the event files in the directory 
# files from all_event_datafiles.csv will be inserted into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('all_event_datafiles.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length','level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

#### View first 5 rows of the data

In [4]:
df = pd.read_csv("all_event_datafiles.csv")
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


#### Check the number of rows in your csv file

In [5]:
with open('all_event_datafiles.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6832


#### Create a Cluster and Session connection to begin query

In [6]:
# Make connection to a Cassandra instance on local machine (127.0.0.1)

try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifyks
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor': 1}"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('sparkifyks')
except Exception as e:
    print(e)

#### Let's define a function to convert the returned query results into Pandas dataframe for better presentation of the returned query results.

In [9]:
def create_dataframe(colnames, rows):
    """
    Converts results of each query into a Pandas dataframe
    
    args:
        colnames: column names of the returned query results
        rows: the returned query results
        
    returns:
        a dataframe of the query result
    
    """
    return pd.DataFrame(rows, columns=colnames)

## QUERY 1:
* `Get the artist, song title and song's length for sessionId = 338, and itemInSession  = 4`

#### CREATE TABLE

In [10]:
query = "CREATE TABLE IF NOT EXISTS artist_song_detail_by_session_and_item"
query = query + "(sessionId int,\
                  itemInSession int,\
                  artist text,\
                  song text, \
                  length float,\
                  PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### EXTRACT, TRANSFORM AND LOAD INTO TABLE

In [11]:
file = 'all_event_datafiles.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO artist_song_detail_by_session_and_item (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT DATA FROM TABLE

In [12]:
# Get the artist, song title and song's length for sessionId = 338, and itemInSession  = 4

query = """SELECT 
               artist, song, length 
           FROM 
               artist_song_detail_by_session_and_item 
           WHERE 
               sessionId=338 AND 
               itemInSession=4
         """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# Display the returned results
session.row_factory = create_dataframe
artist_song_by_session_and_item_df = rows._current_rows
artist_song_by_session_and_item_df

[Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)]

## QUERY 2: 
* `Get the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182`

#### CREATE TABLE

In [13]:
query = "CREATE TABLE IF NOT EXISTS user_detail_by_session"
query = query + "(userId INT,\
            sessionId INT,\
            itemInSession INT,\
            artist TEXT,\
            song TEXT,\
            firstName TEXT,\
            lastName TEXT,\
            PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### EXTRACT, TRANSFORM AND LOAD INTO TABLE

In [14]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO user_detail_by_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### SELECT DATA FROM TABLE

In [15]:
# Get the name of artist, song (sorted by itemInSession) and user (first and last name)
# for userid = 10, sessionid = 182

query = """SELECT 
               artist, song, firstName, lastName
           FROM 
               user_detail_by_session
           WHERE
               userId=10 AND
               sessionId=182
           ORDER BY 
               itemInSession
       """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# Display the returned results
session.row_factory = create_dataframe
detail_user_by_session_df = rows._current_rows
detail_user_by_session_df

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## QUERY 3: 
* `Get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'`

#### CREATE TABLE

In [16]:
query = "CREATE TABLE IF NOT EXISTS user_detail_by_song"
query = query + "(song TEXT,\
                  userId INT,\
                  firstName TEXT,\
                  lastName TEXT,\
                  PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### EXTRACT, TRANSFORM AND LOAD INTO TABLE

In [17]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO user_detail_by_song (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### SELECT DATA FROM TABLE

In [18]:
# Get every user name (first and last) in my music app history who listened 
# to the song 'All Hands Against His Own

query = """SELECT 
               firstName, lastName
           FROM 
               user_detail_by_song
           WHERE
               song='All Hands Against His Own'
       """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# Display the returned results
session.row_factory = create_dataframe
user_detail_by_song_df = rows._current_rows
user_detail_by_song_df

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [19]:
## TO-DO: Drop the table before closing out the sessions
query1 = """DROP TABLE artist_song_detail_by_session_and_item"""
query2 = """DROP TABLE user_detail_by_session"""
query3 = """DROP TABLE user_detail_by_song"""

try:
    session.execute(query1)
    session.execute(query2)
    session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()